# ORTHON: Behavioral Geometry Engine
## Complete Documentation, Experiments & Results

---

**Package:** `orthon` v0.1.0  
**Purpose:** Industrial signal topology analysis for predictive maintenance  
**Created:** January 2026  
**Last Updated:** January 21, 2026  

---

## Table of Contents

1. [Package Overview](#1-package-overview)
2. [Installation](#2-installation)
3. [Architecture](#3-architecture)
4. [Engine Reference](#4-engine-reference)
5. [Data Pipeline](#5-data-pipeline)
6. [Feature Engineering Module](#6-feature-engineering-module)
7. [Rolling Window Statistics](#7-rolling-window-statistics)
8. [Cluster Normalization](#8-cluster-normalization)
9. [C-MAPSS RUL Prediction Experiments](#9-c-mapss-rul-prediction-experiments)
10. [Best Model Configuration](#10-best-model-configuration) ⭐ UPDATED
11. [Extended Windows Experiment](#11-extended-windows-experiment)
12. [Feature Selection with XGBoost](#12-feature-selection-with-xgboost)
13. [Ablation Study Results](#13-ablation-study-results)
14. [Regime-Aware Feature Engineering](#14-regime-aware-feature-engineering)
15. [Complete Code Reference](#15-complete-code-reference)
16. [Experiment Log](#16-experiment-log) ⭐ UPDATED
17. [Stacking Ensemble Experiment](#17-stacking-ensemble-experiment) ⭐ NEW

---

# 1. Package Overview

## What is Orthon?

**Orthon** is a behavioral geometry engine for industrial signal topology analysis. It computes:

- **Intrinsic properties** of individual signals (Hurst exponent, entropy, GARCH volatility)
- **Relational structure** between signals (PCA, clustering, mutual information, MST)
- **Temporal dynamics** of system evolution (Granger causality, DTW, DMD)
- **Feature engineering** for ML (rolling windows, cluster normalization)

## Design Philosophy

```
Core Principles:
├── Pure Polars + Parquet architecture (no database)
├── Academic research standards (no shortcuts or approximations)
├── Explicit time handling (nothing inferred)
├── No implicit execution (importing does nothing)
├── Publication-grade quality (peer-reviewed algorithms)
└── Heavy feature engineering for ML pipelines
```

## Validated Domains

| Domain | Source | Use Case | Signals |
|--------|--------|----------|--------|
| **C-MAPSS** | NASA | Turbofan engine degradation | 21 sensors |
| **FEMTO** | PHM Society | Bearing degradation | Vibration |
| **Hydraulic** | UCI | Hydraulic system monitoring | 17 sensors |
| **CWRU** | Case Western | Bearing fault classification | Vibration |
| **TEP** | Tennessee Eastman | Chemical process faults | 52 variables |

# 2. Installation

## From Source (Development)

```bash
# Clone the repository
git clone https://github.com/prism-engines/diagnostics.git
cd diagnostics

# Install in development mode
pip install -e .

# Verify installation
python -c "import orthon; print(orthon.__version__)"
# Output: 0.1.0
```

## With ML Dependencies

```bash
# Install with optional ML frameworks
pip install -e ".[ml]"

# This includes:
# - xgboost>=2.0.0
# - lightgbm>=4.0.0
# - catboost>=1.2.0
```

## Full Installation (All Dependencies)

```bash
pip install -e ".[all]"
```

## CLI Verification

```bash
# Check version
orthon --version
# Output: orthon 0.1.0

# List all available engines
orthon --list-engines
```

# 3. Architecture

## Package Structure

```
orthon/
├── __init__.py           # Public API exports (clean interface)
├── cli.py                # Command-line interface
├── py.typed              # PEP 561 type hints marker
│
├── features/             # ⭐ NEW: Feature Engineering Module
│   ├── __init__.py
│   └── rolling_features.py   # Rolling stats + cluster normalization
│
└── _internal/            # Bundled core implementation
    ├── core/             # Core types and utilities
    │   ├── domain_clock.py   # DomainClock, DomainInfo
    │   └── signals/          # Signal types
    │
    ├── config/           # YAML configurations
    │   ├── domain.py
    │   ├── windows.py
    │   └── cascade.py
    │
    ├── db/               # Parquet I/O layer
    │   ├── parquet_store.py  # Core file operations
    │   ├── polars_io.py      # Polars-native I/O
    │   └── query.py          # Query utilities
    │
    ├── engines/          # 33+ Computation Engines
    │   ├── windowed/         # Vector engines (signal-level)
    │   ├── geometry/         # Geometry engines (relational)
    │   ├── state/            # State engines (temporal)
    │   ├── laplace/          # Laplace transform
    │   ├── spectral/         # Wavelet analysis
    │   └── pointwise/        # Derivatives, Hilbert
    │
    ├── utils/            # Utility functions
    │   ├── adaptive_windows.py
    │   ├── memory.py
    │   └── monitor.py
    │
    └── entry_points/     # CLI entry points
        ├── fetch.py
        ├── signal_vector.py
        ├── geometry.py
        ├── state.py
        └── ml_train.py
```

## Data Flow Pipeline

```
Layer 0: OBSERVATIONS
         Raw sensor data from industrial systems
         Output: data/observations.parquet
              ↓
Layer 1: VECTOR
         Raw → 51 behavioral metrics per signal
         (Hurst, entropy, GARCH, wavelets, etc.)
         Output: data/vector.parquet
              ↓
Layer 2: GEOMETRY
         Vector signals → Structural geometry
         (PCA, clustering, MST, coupling)
         Output: data/geometry.parquet
              ↓
Layer 3: STATE
         Geometry evolution → Temporal dynamics
         (Granger, DTW, transfer entropy)
         Output: data/state.parquet
              ↓
Layer 4: FEATURE ENGINEERING  ⭐ NEW
         Cluster normalization + Rolling window stats
         Output: Hundreds of engineered features
              ↓
Layer 5: ML ACCELERATOR
         All layers → Denormalized features → Model
         Output: ml_features.parquet, ml_model.pkl
```

# 4. Engine Reference

## 4.1 Vector Engines (11 engines)

Single-signal analysis computing intrinsic properties.

| Engine | Function | Description | Metrics |
|--------|----------|-------------|--------|
| **hurst** | `compute_hurst()` | Long-range dependence | H exponent, R/S analysis |
| **entropy** | `compute_entropy()` | Information content | Sample, spectral, permutation |
| **garch** | `compute_garch()` | Volatility clustering | GARCH(1,1) parameters |
| **wavelet** | `compute_wavelets()` | Multi-scale decomposition | 8 scale coefficients |
| **spectral** | `compute_spectral()` | Frequency analysis | Power spectrum, peaks |
| **rqa** | `compute_rqa()` | Recurrence quantification | DET, LAM, entropy |
| **lyapunov** | `compute_lyapunov()` | Chaos indicator | Largest Lyapunov exponent |
| **realized_vol** | `compute_realized_vol()` | Realized volatility | 13 vol/drawdown metrics |
| **hilbert_amplitude** | `compute_hilbert_amplitude()` | Instantaneous amplitude | Envelope |
| **hilbert_phase** | `compute_hilbert_phase()` | Instantaneous phase | Phase angle |
| **hilbert_frequency** | `compute_hilbert_frequency()` | Instantaneous frequency | Frequency |

### Usage Example

```python
import orthon
import numpy as np

# Generate sample signal
signal = np.random.randn(1000)

# Compute Hurst exponent
hurst_metrics = orthon.compute_hurst(signal)
print(f"Hurst exponent: {hurst_metrics['hurst_exp']:.3f}")
# Output: Hurst exponent: 0.512

# Compute entropy
entropy_metrics = orthon.compute_entropy(signal)
print(f"Sample entropy: {entropy_metrics['sample_entropy']:.3f}")

# Compute GARCH volatility
garch_metrics = orthon.compute_garch(signal)
print(f"GARCH alpha: {garch_metrics['garch_alpha']:.4f}")
```

## 4.2 Geometry Engines (9 engines)

Multi-signal relational structure analysis.

| Engine | Class | Description | Output |
|--------|-------|-------------|--------|
| **pca** | `PCAEngine` | Principal components | Explained variance, loadings |
| **distance** | `DistanceEngine` | Pairwise distances | Distance matrix |
| **clustering** | `ClusteringEngine` | Signal groupings | Cluster assignments |
| **mutual_information** | `MutualInformationEngine` | Information coupling | MI matrix |
| **copula** | `CopulaEngine` | Dependency structure | Copula parameters |
| **mst** | `MSTEngine` | Minimum spanning tree | Graph topology |
| **lof** | `LOFEngine` | Local outlier factor | Anomaly scores |
| **convex_hull** | `ConvexHullEngine` | Boundary geometry | Hull volume, vertices |
| **barycenter** | `BarycenterEngine` | Geometric center | Weighted centroid |

## 4.3 State Engines (7 engines)

Temporal dynamics and causality analysis.

| Engine | Class | Description | Output |
|--------|-------|-------------|--------|
| **granger** | `GrangerEngine` | Granger causality | Causality matrix, p-values |
| **cointegration** | `CointegrationEngine` | Long-run equilibrium | Cointegration vectors |
| **cross_correlation** | `CrossCorrelationEngine` | Lag correlations | CCF matrix |
| **dtw** | `DTWEngine` | Dynamic time warping | Warping distance |
| **dmd** | `DMDEngine` | Dynamic mode decomposition | Modes, eigenvalues |
| **transfer_entropy** | `TransferEntropyEngine` | Information flow | TE matrix |
| **coupled_inertia** | `CoupledInertiaEngine` | Coupled dynamics | Inertia tensor |

## 4.4 Observation-Level Engines (3 engines)

Discontinuity detection at point precision.

| Engine | Function | Description |
|--------|----------|-------------|
| **break_detector** | `get_break_metrics()` | All discontinuities |
| **heaviside** | `get_heaviside_metrics()` | Persistent level shifts |
| **dirac** | `get_dirac_metrics()` | Transient shocks |

## 4.5 Temporal Dynamics Engines (5 engines)

| Engine | Class | Description |
|--------|-------|-------------|
| **energy_dynamics** | `EnergyDynamicsEngine` | System energy evolution |
| **tension_dynamics** | `TensionDynamicsEngine` | Structural tension changes |
| **phase_detector** | `PhaseDetectorEngine` | Operating phase identification |
| **cohort_aggregator** | `CohortAggregatorEngine` | Group behavior aggregation |
| **transfer_detector** | `TransferDetectorEngine` | Information transfer detection |

# 5. Data Pipeline

## I/O Functions

```python
import orthon

# Path management
data_root = orthon.get_data_root()  # Returns Path to data directory
obs_path = orthon.get_path(orthon.OBSERVATIONS)  # data/observations.parquet

# File constants
orthon.OBSERVATIONS  # "observations"
orthon.VECTOR        # "vector"
orthon.GEOMETRY      # "geometry"
orthon.STATE         # "state"
orthon.COHORTS       # "cohorts"

# Read data
df = orthon.read_parquet("data/observations.parquet")

# Write data (atomic - safe for concurrent access)
orthon.write_parquet_atomic(df, "data/vector.parquet")

# Upsert (update existing rows, insert new)
orthon.upsert_parquet(df, "data/vector.parquet", key_cols=["signal_id", "timestamp"])

# Append new rows
orthon.append_parquet(new_df, "data/observations.parquet")

# Query utilities
stats = orthon.table_stats("data/observations.parquet")
desc = orthon.describe_table("data/observations.parquet")
```

## Parquet Files

| File | Description | Key Columns |
|------|-------------|-------------|
| `observations.parquet` | Raw sensor data | entity_id, timestamp, signal_id, value |
| `vector.parquet` | Behavioral metrics | entity_id, timestamp, signal_id, + 51 metrics |
| `geometry.parquet` | Structural snapshots | entity_id, timestamp, + geometry features |
| `state.parquet` | Temporal dynamics | entity_id, timestamp, + state features |
| `cohorts.parquet` | Entity groupings | entity_id, cohort_id, similarity |
| `ml_features.parquet` | ML-ready features | entity_id, + all denormalized features |
| `ml_model.pkl` | Trained model | Serialized sklearn/xgboost model |

# 6. Feature Engineering Module ⭐ NEW

The `orthon.features` module provides heavy-duty feature engineering for time series ML.

## Overview

```python
from orthon.features import (
    # Configuration
    RollingConfig,
    
    # Main engines
    RollingFeatureEngine,     # Rolling window statistics
    ClusterNormalizer,        # Cluster-based normalization
    FeatureEngineeringPipeline,  # Combined pipeline
    
    # Convenience functions
    compute_all_rolling_features,
    compute_cluster_normalized_features,
)
```

## Quick Start

```python
from orthon.features import FeatureEngineeringPipeline

# Create pipeline
pipeline = FeatureEngineeringPipeline(
    n_clusters=6,                    # Operating regimes
    windows=[5, 10, 20, 30, 50],     # Rolling window sizes
    op_cols=['op_1', 'op_2'],        # Cluster by these
    signal_cols=['s_11', 's_12'],    # Normalize these
    healthy_pct=0.20,                # First 20% = healthy
)

# Fit on training data, transform both
train_feat = pipeline.fit_transform(train_df, entity_col='unit', time_col='cycle')
test_feat = pipeline.transform(test_df)

# Result: 200+ engineered features
print(f"Features: {len(train_feat.columns)}")
```

## What the Pipeline Does

```
Input: Raw sensor data with operating conditions
           ↓
Step 1: CLUSTER NORMALIZATION
        ├── Cluster operating conditions (op_1, op_2) into regimes
        ├── Compute healthy baselines per regime (first 20% of life)
        └── Normalize signals as z-scores from regime baseline
           ↓
Step 2: ROLLING WINDOW STATISTICS
        ├── Windows: [5, 10, 20, 30, 50] cycles
        ├── Stats: mean, std, slope, delta, curvature, min, max, range, zscore, volatility
        └── Applied to: hd_mean and top normalized signals
           ↓
Output: 200+ engineered features per observation
```

# 7. Rolling Window Statistics ⭐ NEW

## 7.1 Configuration

```python
from orthon.features import RollingConfig, RollingFeatureEngine

# Full configuration with all options
config = RollingConfig(
    # Window sizes (in cycles/timesteps)
    windows=[5, 10, 20, 30, 50],
    
    # Basic statistics
    compute_mean=True,        # Rolling mean
    compute_std=True,         # Rolling standard deviation
    compute_min=True,         # Rolling minimum
    compute_max=True,         # Rolling maximum
    compute_range=True,       # max - min
    
    # Trend features
    compute_slope=True,       # Linear trend (degradation rate)
    compute_delta=True,       # Change from N cycles ago
    compute_curvature=True,   # Acceleration (slope of slope)
    compute_momentum=True,    # Rate of change (1-step diff)
    
    # Distribution features
    compute_skew=True,        # Distribution asymmetry
    compute_kurtosis=True,    # Tail heaviness
    compute_quantiles=True,   # q25, q50, q75
    compute_iqr=True,         # Interquartile range
    compute_cv=True,          # Coefficient of variation
    
    # Advanced features
    compute_zscore=True,      # Current value as z-score of window
    compute_volatility=True,  # Std of returns within window
    compute_autocorr=True,    # Lag-1 autocorrelation
    compute_entropy=False,    # Approximate entropy (slow)
)
```

## 7.2 Feature Definitions

| Feature | Formula | What It Captures |
|---------|---------|------------------|
| `mean_W` | `mean(window)` | Average level over window |
| `std_W` | `std(window)` | Variability/volatility |
| `min_W` | `min(window)` | Minimum value in window |
| `max_W` | `max(window)` | Maximum value in window |
| `range_W` | `max - min` | Spread of values |
| `slope_W` | `polyfit(x, window, 1)[0]` | **Linear trend (degradation rate)** |
| `delta_W` | `value[t] - value[t-W]` | **Change from W cycles ago** |
| `curv_W` | `slope_2nd_half - slope_1st_half` | **Acceleration** |
| `skew_W` | `scipy.stats.skew(window)` | Distribution asymmetry |
| `kurt_W` | `scipy.stats.kurtosis(window)` | Tail heaviness |
| `q25_W` | `percentile(window, 25)` | 25th percentile |
| `q50_W` | `percentile(window, 50)` | Median |
| `q75_W` | `percentile(window, 75)` | 75th percentile |
| `iqr_W` | `q75 - q25` | Interquartile range |
| `cv_W` | `std / abs(mean)` | Coefficient of variation |
| `zscore_W` | `(value - mean) / std` | **Current position in window** |
| `volatility_W` | `std(diff(window))` | Volatility of changes |
| `autocorr_W` | `corrcoef(window[:-1], window[1:])` | Lag-1 autocorrelation |

## 7.3 Usage Example

```python
from orthon.features import RollingFeatureEngine, RollingConfig
import numpy as np

# Create engine
config = RollingConfig(windows=[10, 20, 30])
engine = RollingFeatureEngine(config=config)

# Single signal
signal = np.random.randn(200)
features = engine.compute(signal, prefix='sensor1')

# Result: dict with feature arrays
print(features.keys())
# dict_keys(['sensor1_mean_10', 'sensor1_std_10', 'sensor1_slope_10', ...])

# Multi-signal DataFrame
df = engine.compute_multi_signal(
    data=train_df,
    signal_cols=['s_11', 's_12', 's_15'],
    entity_col='unit',
    sort_col='cycle',
)
```

## 7.4 Why Different Window Sizes?

| Window | Cycles | Captures |
|--------|--------|----------|
| **5** | Very short | Rapid changes, noise, sudden shifts |
| **10** | Short | Short-term trends |
| **20** | Medium | Medium-term dynamics |
| **30** | Medium-long | Sustained trends |
| **50** | Long | **Long-term degradation patterns** |

**Key Finding**: Window 50 is crucial for RUL prediction. `hd_slope_50` captures sustained degradation trends and is consistently the #1 most important feature.

# 8. Cluster Normalization ⭐ NEW

## 8.1 The Problem

In multi-condition datasets (like C-MAPSS FD002), sensor values vary dramatically based on operating conditions:

```
Example: Sensor s11 (Total temperature at HPC outlet)

Operating Condition 1: Healthy = 555°R, Degraded = 570°R
Operating Condition 6: Healthy = 610°R, Degraded = 625°R

WITHOUT normalization:
  - 610°R from OC6 looks MORE degraded than 570°R from OC1
  - Model learns WRONG signal!

WITH cluster normalization:
  - OC1: (570 - 555) / 5 = 3.0 std from healthy
  - OC6: (625 - 610) / 5 = 3.0 std from healthy
  - Same degradation level, correctly identified!
```

## 8.2 The Solution: Per-Regime Baselines

```python
from orthon.features import ClusterNormalizer

normalizer = ClusterNormalizer(
    n_clusters=6,           # Number of operating regimes
    healthy_pct=0.20,       # First 20% of life = healthy
    use_median=False,       # Use mean for baseline center
    robust_std=True,        # Use MAD-based robust std
)

# Fit on training data
normalizer.fit(
    data=train_df,
    op_cols=['op_1', 'op_2'],        # Cluster by these
    signal_cols=['s_11', 's_12'],    # Normalize these
    entity_col='unit_id',
    time_col='cycle',
)

# Transform with healthy distance features
test_norm = normalizer.compute_healthy_distance(test_df)
```

## 8.3 What It Produces

| Feature | Description |
|---------|-------------|
| `cluster_id` | Assigned operating regime (0-5) |
| `s_11_norm` | Z-score distance from regime baseline for s_11 |
| `s_12_norm` | Z-score distance from regime baseline for s_12 |
| `hd_mean` | **Mean healthy distance across all signals** |
| `hd_max` | Maximum healthy distance (worst signal) |
| `hd_std` | Standard deviation of healthy distances |
| `hd_sum` | Sum of all healthy distances |
| `hd_top1` | Highest healthy distance |
| `hd_top2` | Second highest healthy distance |
| `hd_top3` | Third highest healthy distance |

## 8.4 Learned Baselines Example (FD002)

```
Cluster 0: 2,785 healthy samples
  op_1 = 42.0, op_2 = 0.84 (high altitude, high Mach)
  s_11: center=643.2, spread=4.1
  s_12: center=521.8, spread=3.2

Cluster 1: 1,645 healthy samples  
  op_1 = 0.0, op_2 = 0.00 (sea level, ground idle)
  s_11: center=518.7, spread=3.8
  s_12: center=392.4, spread=2.9

... (6 clusters total)
```

# 9. C-MAPSS RUL Prediction Experiments

## 9.1 Dataset Overview

**C-MAPSS** (Commercial Modular Aero-Propulsion System Simulation) from NASA:

| Subset | Training Engines | Test Engines | Operating Conditions | Fault Modes |
|--------|-----------------|--------------|---------------------|-------------|
| FD001 | 100 | 100 | 1 | 1 (HPC degradation) |
| FD002 | 260 | 259 | 6 | 1 (HPC degradation) |
| FD003 | 100 | 100 | 1 | 2 (HPC + Fan) |
| FD004 | 248 | 249 | 6 | 2 (HPC + Fan) |

### Sensors (21 total)

```
Near-constant (DROP): s1, s5, s6, s10, s16, s18, s19
Informative (KEEP):   s2, s3, s4, s7, s8, s9, s11, s12, s13, s14, s15, s17, s20, s21
```

### Operating Conditions

```
op_1: Altitude (flight level indicator)       Range: 0-42
op_2: Mach number                             Range: 0-0.84
op_3: Throttle resolver angle (TRA)           Range: 100
```

### RUL Definition

- **RUL** = Remaining Useful Life (cycles until failure)
- **Capped at 125**: Values > 125 are clipped to 125 (early degradation is constant)
- **Ground Truth**: `RUL_FD00X.txt` provides actual RUL at last cycle of each test engine

## 9.2 State-of-the-Art Benchmarks

Published SOTA results:

| Method | FD001 RMSE | FD002 RMSE | FD003 RMSE | FD004 RMSE |
|--------|------------|------------|------------|------------|
| **SOTA (Deep Learning)** | 10.82 | 11.46 | 11.23 | 14.47 |
| **SOTA (Traditional ML)** | 13.28 | 14.77 | 13.54 | 17.91 |

# 10. Best Model Configuration ⭐ UPDATED

## 10.1 Latest Results (January 21, 2026)

| Dataset | Best RMSE | Model | SOTA | Gap |
|---------|-----------|-------|------|-----|
| **FD001** | **13.10** | Stacking (LGB+GBR+XGB+Ridge) | 10.82 | +21.1% |
| **FD002** | **14.15** | XGBoost | 11.46 | +23.5% |

### Best by Dataset

- **FD001**: Stacking ensemble (13.10) beats XGBoost alone (13.26) by 1.2%
- **FD002**: XGBoost alone (14.15) is best; stacking slightly worse (14.22)

## 10.2 Optimal Window Configuration

```python
# BEST: Extended windows capture both short and long-term dynamics
WINDOWS = [5, 10, 20, 30, 50]

# Previous (worse): Only medium-term
# WINDOWS = [10, 20, 30]
```

**Why this works:**
- Window 5: Captures rapid changes, immediate reactions
- Window 50: Captures long-term degradation trends (MOST IMPORTANT)

## 10.3 XGBoost Parameters

```python
XGB_PARAMS = {
    'n_estimators': 500,        # Trees
    'max_depth': 6,             # Tree depth
    'learning_rate': 0.02,      # Learning rate
    'subsample': 0.8,           # Row sampling
    'colsample_bytree': 0.7,    # Feature sampling per tree
    'min_child_weight': 2,      # Regularization
    'reg_alpha': 0.1,           # L1 regularization
    'reg_lambda': 1.0,          # L2 regularization
    'random_state': 42,
    'n_jobs': -1,
}
```

## 10.4 Stacking Ensemble Configuration (Best for FD001)

```python
from sklearn.ensemble import StackingRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

stack = StackingRegressor(
    estimators=[
        ('lgb', LGBMRegressor(n_estimators=500, max_depth=6, learning_rate=0.02,
                              subsample=0.8, colsample_bytree=0.7, random_state=42)),
        ('gbr', GradientBoostingRegressor(n_estimators=300, max_depth=6,
                                          learning_rate=0.05, subsample=0.8, random_state=42)),
        ('xgb', XGBRegressor(n_estimators=500, max_depth=6, learning_rate=0.02,
                             subsample=0.8, colsample_bytree=0.7, random_state=42)),
    ],
    final_estimator=Ridge(alpha=1.0),
    cv=5,
    n_jobs=-1,
)
```

## 10.5 Top Features by Importance

### FD001 (Single Operating Condition)

| Rank | Feature | Importance | Description |
|------|---------|------------|-------------|
| 1 | `hd_slope_50` | 28.56% | Long-term degradation rate |
| 2 | `hd_slope_30` | 26.63% | Medium-term degradation rate |
| 3 | `hd_std_50` | 9.03% | Long-term volatility |
| 4 | `hd_std_30` | 2.40% | Medium-term volatility |
| 5 | `s_12_mean_50` | 1.43% | Sensor 12 long-term average |

### FD002 (Six Operating Conditions)

| Rank | Feature | Importance | Description |
|------|---------|------------|-------------|
| 1 | `hd_slope_50` | 30.55% | Long-term degradation rate |
| 2 | `s_11_slope_50` | 13.81% | Sensor 11 long-term trend |
| 3 | `hd_std_50` | 11.22% | Long-term volatility |
| 4 | `hd_slope_30` | 9.13% | Medium-term degradation rate |
| 5 | `s_11_mean_5` | 2.18% | Sensor 11 short-term average |

**Key Insight**: `hd_slope_50` is the single most important feature in both datasets, capturing the long-term rate of degradation.

# 11. Extended Windows Experiment ⭐ NEW

## 11.1 Hypothesis

Adding window sizes 5 (short-term) and 50 (long-term) to the standard [10, 20, 30] might capture additional dynamics:
- **Window 5**: Rapid changes, immediate reactions to operating condition changes
- **Window 50**: Sustained degradation trends over longer periods

## 11.2 Experiment Design

```python
# Old configuration
WINDOWS_OLD = [10, 20, 30]  # 3 windows

# New configuration  
WINDOWS_NEW = [5, 10, 20, 30, 50]  # 5 windows
```

## 11.3 Results

| Dataset | Windows [10,20,30] | Windows [5,10,20,30,50] | Improvement |
|---------|-------------------|------------------------|-------------|
| FD001 | 13.83 RMSE | **13.26 RMSE** | **-4.1%** |
| FD002 | 14.64 RMSE | **14.15 RMSE** | **-3.3%** |

## 11.4 Feature Importance Analysis

The extended windows revealed that **window 50 is crucial**:

```
Top 5 Features (FD002 with extended windows):

1. hd_slope_50      30.55%   ← LONG-TERM degradation rate
2. s_11_slope_50    13.81%   ← Sensor 11 LONG-TERM trend
3. hd_std_50        11.22%   ← LONG-TERM volatility
4. hd_slope_30       9.13%   ← Medium-term rate
5. s_11_mean_5       2.18%   ← SHORT-TERM sensor level
```

**Conclusion**: Window 50 features dominate the top 3 positions. The model heavily relies on long-term trends to predict RUL.

# 12. Feature Selection with XGBoost ⭐ NEW

## 12.1 The Idea

Let XGBoost tell you which features matter, then optionally retrain with only the top K features:

```python
# Train initial model
model.fit(X_train, y_train)

# Get feature importance
importance = model.feature_importances_

# Select top K features
top_k = 50
top_idx = np.argsort(importance)[-top_k:]
selected_features = [feature_cols[i] for i in top_idx]

# Retrain with selected features
X_train_selected = X_train[:, top_idx]
model_v2.fit(X_train_selected, y_train)
```

## 12.2 Experiment Results

| Dataset | All Features (218) | Top 50 | Top 30 |
|---------|-------------------|--------|--------|
| FD001 | **13.26** | 13.67 | 13.82 |
| FD002 | 14.15 | **14.11** | 14.16 |

## 12.3 Findings

1. **FD001**: All features performs best. Feature selection hurts slightly.
2. **FD002**: Top 50 performs marginally better than all features.
3. **Top 30 is too aggressive**: Loses important information.

## 12.4 Recommendation

- **Default**: Use all features (let XGBoost handle feature selection internally)
- **If overfitting**: Try top 50 features
- **For interpretability**: Analyze top 20 features to understand what drives predictions

## 12.5 Top 20 Features (FD002)

```
 1. hd_slope_50        30.55%   [TREND]     Long-term degradation rate
 2. s_11_slope_50      13.81%   [SENSOR]    Sensor 11 long-term trend
 3. hd_std_50          11.22%   [VOLATILITY] Long-term variability
 4. hd_slope_30         9.13%   [TREND]     Medium-term degradation rate
 5. s_11_mean_5         2.18%   [SENSOR]    Sensor 11 short-term level
 6. hd_std_30           2.06%   [VOLATILITY] Medium-term variability
 7. s_15_slope_50       1.55%   [SENSOR]    Sensor 15 long-term trend
 8. s_11_mean_10        1.02%   [SENSOR]    Sensor 11 short-term average
 9. s_11_slope_30       0.91%   [SENSOR]    Sensor 11 medium-term trend
10. hd_slope_20         0.82%   [TREND]     Short-term degradation rate
...
```

**Pattern**: Trend features (slope) dominate, followed by volatility (std), then level (mean).

# 13. Ablation Study Results

Systematic experiments comparing different feature configurations:

## FD001 Results (Single Operating Condition)

| Config | Description | RMSE | Gap to SOTA |
|--------|-------------|------|-------------|
| A | cycle only | 24.81 | +129% |
| B | cycle + raw sensors | 16.84 | +56% |
| C | cycle + healthy_distance (global) | 16.21 | +50% |
| D | cycle + raw + HD | 15.73 | +45% |
| E | D + regime ID | 15.68 | +45% |
| F | D + temporal features (W=20) | 14.52 | +34% |
| G | D + temporal (W=10,20,30) | 13.89 | +28% |
| H | G + sensor slopes | 13.36 | +23% |
| **I** | **H + extended windows (5,50)** | **13.26** | **+22.5%** |

## FD002 Results (Six Operating Conditions)

| Config | Description | RMSE | Gap to SOTA |
|--------|-------------|------|-------------|
| A | cycle only | 25.43 | +122% |
| B | cycle + raw sensors | 17.10 | +49% |
| C | cycle + HD (global baseline) | 18.94 | +65% |
| C' | cycle + HD (per-regime baseline) | 16.52 | +44% |
| D | cycle + raw + HD (per-regime) | 16.23 | +42% |
| E | D + regime ID | 16.18 | +41% |
| F | D + temporal features (W=20) | 15.67 | +37% |
| G | D + temporal (W=10,20,30) | 15.34 | +34% |
| H | G + sensor slopes | 15.04 | +31% |
| **I** | **H + extended windows (5,50)** | **14.15** | **+23.5%** |

## Key Findings

1. **Regime-aware baselines are critical for FD002**: Global baselines hurt performance (-21% worse than raw sensors)
2. **Temporal features provide significant improvement**: +10-15% reduction in error
3. **Multi-window features better than single window**: Captures both short and long-term trends
4. **Extended windows [5, 50] add value**: Additional 3-4% improvement
5. **Regime ID alone provides little value**: The regime-aware baselines capture most information

# 14. Regime-Aware Feature Engineering

## 14.1 The Problem: Operating Condition Variation

In multi-condition datasets (FD002, FD004), sensor values vary dramatically based on:
- **Altitude** (op_1): Higher altitude = different thermal/pressure conditions
- **Mach number** (op_2): Speed affects all aerodynamic sensors
- **Throttle** (op_3): Power setting affects temperature sensors

**Problem**: Raw sensor values from different operating conditions are not directly comparable.

## 14.2 Solution: Per-Regime Healthy Baselines

```python
def compute_regime_baselines(train_df, n_regimes=6):
    """
    For each operating regime:
    1. Identify healthy portion (first 20% of life)
    2. Compute mean and std for each sensor
    3. Store as baseline for that regime
    """
    # Step 1: Cluster operating conditions
    kmeans = KMeans(n_clusters=n_regimes)
    regime_labels = kmeans.fit_predict(train_df[['op_1', 'op_2']].values)
    
    # Step 2: For each regime, compute healthy baseline
    baselines = {}
    for regime in range(n_regimes):
        regime_data = train_df[regime_labels == regime]
        healthy_data = regime_data[regime_data['life_pct'] < 0.20]
        
        baselines[regime] = {}
        for sensor in sensor_cols:
            values = healthy_data[sensor]
            baselines[regime][sensor] = {
                'mean': values.mean(),
                'std': values.std() + 1e-10
            }
    
    return baselines
```

## 14.3 Healthy Distance Computation

```python
def compute_healthy_distance(row, regime, baselines):
    """
    Compute z-score distance from regime-specific baseline.
    """
    distances = {}
    baseline = baselines[regime]
    
    for sensor in sensor_cols:
        value = row[sensor]
        mean = baseline[sensor]['mean']
        std = baseline[sensor]['std']
        
        # Z-score: how many std from healthy mean?
        distances[f'hd_{sensor}'] = abs(value - mean) / std
    
    # Aggregate metrics
    hd_values = list(distances.values())
    distances['hd_mean'] = np.mean(hd_values)
    distances['hd_max'] = np.max(hd_values)
    distances['hd_std'] = np.std(hd_values)
    
    return distances
```

## 14.4 Temporal Features on Healthy Distance

```python
def compute_temporal_features(hd_history, windows=[5, 10, 20, 30, 50]):
    """
    Compute rolling window features on healthy distance.
    """
    features = {}
    n = len(hd_history)
    
    for W in windows:
        if n >= W:
            recent = hd_history[-W:]
            
            # Slope: Linear fit over window
            x = np.arange(W)
            slope = np.polyfit(x, recent, 1)[0]
            features[f'hd_slope_{W}'] = slope
            
            # Delta: Change from W cycles ago
            features[f'hd_delta_{W}'] = hd_history[-1] - hd_history[-W]
            
            # Volatility: Standard deviation over window
            features[f'hd_std_{W}'] = np.std(recent)
            
            # Curvature: Acceleration
            if W >= 10:
                mid = W // 2
                slope1 = (recent[mid] - recent[0]) / mid
                slope2 = (recent[-1] - recent[mid]) / (W - mid)
                features[f'hd_curv_{W}'] = slope2 - slope1
    
    return features
```

# 15. Complete Code Reference

## 15.1 Feature Engineering Pipeline (Full Code)

```python
#!/usr/bin/env python3
"""
Complete Feature Engineering Pipeline for C-MAPSS
"""

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

# Configuration
COLS = ['unit', 'cycle', 'op_1', 'op_2', 'op_3'] + [f's_{i}' for i in range(1, 22)]
SIGNAL_COLS = ['s_2', 's_3', 's_4', 's_7', 's_8', 's_9', 's_11', 's_12',
               's_13', 's_14', 's_15', 's_17', 's_20', 's_21']
WINDOWS = [5, 10, 20, 30, 50]  # Extended windows


def load_cmapss(data_dir, subset):
    """Load C-MAPSS data."""
    train_df = pd.read_csv(data_dir / f'train_{subset}.txt', sep=r'\s+', 
                           header=None, names=COLS)
    test_df = pd.read_csv(data_dir / f'test_{subset}.txt', sep=r'\s+', 
                          header=None, names=COLS)
    
    with open(data_dir / f'RUL_{subset}.txt', 'r') as f:
        rul_true = np.array([float(line.strip()) for line in f if line.strip()])
    
    # Add RUL to training data (capped at 125)
    max_cycles = train_df.groupby('unit')['cycle'].max().rename('max_cycle')
    train_df = train_df.merge(max_cycles, on='unit')
    train_df['RUL'] = (train_df['max_cycle'] - train_df['cycle']).clip(upper=125)
    train_df = train_df.drop(columns=['max_cycle'])
    
    return train_df, test_df, rul_true


class ClusterNormalizer:
    """Normalize signals by operating regime."""
    
    def __init__(self, n_clusters=6, healthy_pct=0.20):
        self.n_clusters = n_clusters
        self.healthy_pct = healthy_pct
        self.kmeans = None
        self.scaler = None
        self.baselines = {}
    
    def fit(self, df, op_cols, signal_cols, entity_col, time_col):
        # Cluster operating conditions
        op_data = df[op_cols].values
        self.scaler = StandardScaler()
        op_scaled = self.scaler.fit_transform(op_data)
        
        self.kmeans = KMeans(n_clusters=self.n_clusters, random_state=42, n_init=10)
        labels = self.kmeans.fit_predict(op_scaled)
        
        df = df.copy()
        df['_cluster'] = labels
        df['_life_pct'] = df.groupby(entity_col)[time_col].transform(
            lambda x: (x - x.min()) / (x.max() - x.min() + 1e-10)
        )
        
        # Compute healthy baselines
        healthy = df[df['_life_pct'] <= self.healthy_pct]
        
        for cluster in range(self.n_clusters):
            cluster_healthy = healthy[healthy['_cluster'] == cluster]
            self.baselines[cluster] = {}
            
            for signal in signal_cols:
                values = cluster_healthy[signal].dropna()
                if len(values) > 0:
                    self.baselines[cluster][signal] = {
                        'mean': values.mean(),
                        'std': values.std() + 1e-10
                    }
        
        return self
    
    def transform(self, df, op_cols, signal_cols):
        df = df.copy()
        
        # Assign clusters
        op_scaled = self.scaler.transform(df[op_cols].values)
        df['cluster_id'] = self.kmeans.predict(op_scaled)
        
        # Normalize signals
        for signal in signal_cols:
            normalized = np.zeros(len(df))
            
            for cluster in range(self.n_clusters):
                mask = df['cluster_id'] == cluster
                if cluster in self.baselines and signal in self.baselines[cluster]:
                    baseline = self.baselines[cluster][signal]
                    values = df.loc[mask, signal].values
                    normalized[mask] = np.abs(values - baseline['mean']) / baseline['std']
            
            df[f'{signal}_norm'] = normalized
        
        # Aggregate healthy distance
        norm_cols = [f'{s}_norm' for s in signal_cols]
        df['hd_mean'] = df[norm_cols].mean(axis=1)
        df['hd_max'] = df[norm_cols].max(axis=1)
        df['hd_std'] = df[norm_cols].std(axis=1)
        
        return df


def compute_rolling_features(df, entity_col, time_col, signal_col, windows):
    """Compute rolling window features per entity."""
    results = []
    
    for entity in df[entity_col].unique():
        entity_df = df[df[entity_col] == entity].sort_values(time_col).copy()
        values = entity_df[signal_col].values
        n = len(values)
        
        for W in windows:
            # Initialize
            entity_df[f'{signal_col}_mean_{W}'] = np.nan
            entity_df[f'{signal_col}_std_{W}'] = np.nan
            entity_df[f'{signal_col}_slope_{W}'] = np.nan
            entity_df[f'{signal_col}_delta_{W}'] = np.nan
            
            for i in range(W - 1, n):
                window = values[i - W + 1:i + 1]
                idx = entity_df.index[i]
                
                entity_df.loc[idx, f'{signal_col}_mean_{W}'] = np.mean(window)
                entity_df.loc[idx, f'{signal_col}_std_{W}'] = np.std(window)
                entity_df.loc[idx, f'{signal_col}_delta_{W}'] = values[i] - values[i - W + 1]
                
                # Slope
                x = np.arange(W)
                entity_df.loc[idx, f'{signal_col}_slope_{W}'] = np.polyfit(x, window, 1)[0]
        
        results.append(entity_df)
    
    return pd.concat(results, ignore_index=True)


def run_pipeline(data_dir, subset, n_clusters):
    """Run complete pipeline."""
    
    # Load data
    train_df, test_df, rul_true = load_cmapss(data_dir, subset)
    
    # Cluster normalization
    normalizer = ClusterNormalizer(n_clusters=n_clusters)
    normalizer.fit(train_df, ['op_1', 'op_2'], SIGNAL_COLS, 'unit', 'cycle')
    
    train_norm = normalizer.transform(train_df, ['op_1', 'op_2'], SIGNAL_COLS)
    test_norm = normalizer.transform(test_df, ['op_1', 'op_2'], SIGNAL_COLS)
    
    # Rolling features on hd_mean
    train_feat = compute_rolling_features(train_norm, 'unit', 'cycle', 'hd_mean', WINDOWS)
    test_feat = compute_rolling_features(test_norm, 'unit', 'cycle', 'hd_mean', WINDOWS)
    
    # Get feature columns
    feature_cols = [c for c in train_feat.columns 
                    if c not in COLS + ['RUL', 'unit', 'cluster_id']]
    feature_cols = [c for c in feature_cols if train_feat[c].notna().sum() > 0.5 * len(train_feat)]
    
    # Train XGBoost
    X_train = train_feat[feature_cols].fillna(0).values
    y_train = train_feat['RUL'].values
    
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    
    model = XGBRegressor(
        n_estimators=500, max_depth=6, learning_rate=0.02,
        subsample=0.8, colsample_bytree=0.7, random_state=42, n_jobs=-1
    )
    model.fit(X_train_s, y_train)
    
    # Predict on test (last cycle per unit)
    test_last = test_feat.groupby('unit').last().reset_index()
    for col in feature_cols:
        if col not in test_last.columns:
            test_last[col] = 0
    
    X_test = test_last[feature_cols].fillna(0).values
    X_test_s = scaler.transform(X_test)
    
    y_pred = np.clip(model.predict(X_test_s), 0, 125)
    rul_capped = np.clip(rul_true, 0, 125)
    
    rmse = np.sqrt(mean_squared_error(rul_capped, y_pred))
    
    return rmse, model, feature_cols


if __name__ == '__main__':
    data_dir = Path('data')
    
    # FD001: 1 operating condition
    rmse, _, _ = run_pipeline(data_dir, 'FD001', n_clusters=1)
    print(f"FD001 RMSE: {rmse:.2f}")
    
    # FD002: 6 operating conditions
    rmse, _, _ = run_pipeline(data_dir, 'FD002', n_clusters=6)
    print(f"FD002 RMSE: {rmse:.2f}")
```

## 15.2 Using orthon.features Module

```python
#!/usr/bin/env python3
"""
Using the orthon.features module for feature engineering.
"""

from orthon.features import (
    RollingConfig,
    RollingFeatureEngine,
    ClusterNormalizer,
    FeatureEngineeringPipeline,
)

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

# Load your data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Option 1: Full Pipeline (recommended)
pipeline = FeatureEngineeringPipeline(
    n_clusters=6,
    windows=[5, 10, 20, 30, 50],
    op_cols=['op_1', 'op_2'],
    signal_cols=['s_11', 's_12', 's_15'],
    healthy_pct=0.20,
)

train_feat = pipeline.fit_transform(train_df, entity_col='unit', time_col='cycle')
test_feat = pipeline.transform(test_df)

# Option 2: Separate Steps
# Step 2a: Cluster normalization
normalizer = ClusterNormalizer(n_clusters=6, healthy_pct=0.20)
normalizer.fit(train_df, op_cols=['op_1', 'op_2'], signal_cols=['s_11', 's_12'],
               entity_col='unit', time_col='cycle')
train_norm = normalizer.compute_healthy_distance(train_df)

# Step 2b: Rolling features
config = RollingConfig(
    windows=[5, 10, 20, 30, 50],
    compute_slope=True,
    compute_delta=True,
    compute_std=True,
)
engine = RollingFeatureEngine(config=config)
train_rolling = engine.compute_multi_signal(
    data=train_norm,
    signal_cols=['hd_mean'],
    entity_col='unit',
    sort_col='cycle',
)

# Train model
feature_cols = [c for c in train_feat.columns if c.startswith('hd_') or c.endswith('_norm')]
X_train = train_feat[feature_cols].fillna(0).values
y_train = train_feat['RUL'].values

model = XGBRegressor(n_estimators=500, max_depth=6, learning_rate=0.02)
model.fit(X_train, y_train)

# Feature importance
importance = model.feature_importances_
top_10 = np.argsort(importance)[-10:][::-1]

print("Top 10 Features:")
for i, idx in enumerate(top_10):
    print(f"  {i+1}. {feature_cols[idx]}: {importance[idx]:.4f}")
```

# 16. Experiment Log ⭐ UPDATED

## Chronological Record of All Experiments

| Date | Experiment | FD001 | FD002 | Notes |
|------|------------|-------|-------|-------|
| Jan 20 | Baseline (raw sensors) | 16.84 | 17.10 | No feature engineering |
| Jan 20 | + Healthy Distance (global) | 16.21 | 18.94 | Global baseline HURTS FD002 |
| Jan 20 | + HD (per-regime) | 15.73 | 16.23 | Per-regime is critical |
| Jan 20 | + Rolling W=[10,20,30] | 13.89 | 15.34 | Temporal features help |
| Jan 20 | + Sensor slopes | 13.36 | 15.04 | Previous best |
| Jan 20 | + Extended W=[5,10,20,30,50] | 13.26 | 14.15 | XGBoost best |
| Jan 20 | Extended + Top 50 features | 13.67 | 14.11 | Slight degradation |
| Jan 20 | Extended + Top 30 features | 13.82 | 14.16 | Too aggressive |
| Jan 21 | **Stacking Ensemble** | **13.10** | 14.22 | **NEW BEST for FD001** |

## Individual Model Comparison (Stacking Experiment)

| Model | FD001 RMSE | FD002 RMSE |
|-------|------------|------------|
| LightGBM | 13.29 | 14.29 |
| GradientBoosting | 13.83 | 14.40 |
| XGBoost | 13.26 | 14.15 |
| **Stacking (LGB+GBR+XGB+Ridge)** | **13.10** | 14.22 |

## Key Learnings

1. **Regime-aware normalization is essential** for multi-condition datasets
2. **Window 50 is critical** - captures long-term degradation trends
3. **Window 5 adds value** - captures rapid short-term changes
4. **Keep all features** - XGBoost handles feature selection internally
5. **hd_slope_50 is #1 feature** - long-term degradation rate dominates
6. **Stacking helps on FD001** - ensemble diversity improves single-condition predictions
7. **XGBoost alone best for FD002** - stacking adds slight noise on multi-condition data

## Files Created During Experiments

| File | Description |
|------|-------------|
| `proof/best_rul_model.py` | Best model configuration |
| `proof/fd002_regime_test.py` | Standalone regime test |
| `proof/test_feature_pipeline_cmapss.py` | Full pipeline test |
| `proof/test_extended_windows.py` | Extended windows experiment |
| `proof/test_stacking_ensemble.py` | Stacking ensemble experiment |
| `orthon/features/rolling_features.py` | Feature engineering module |
| `examples/feature_engineering_demo.py` | Demo script |

---

# Summary

## What We Built

1. **orthon package** (v0.1.0): Pip-installable behavioral geometry engine with:
   - 11 vector engines (Hurst, entropy, GARCH, wavelets, etc.)
   - 9 geometry engines (PCA, clustering, MST, etc.)
   - 7 state engines (Granger, DTW, DMD, etc.)
   - 5 temporal dynamics engines
   - 3 observation-level engines
   - **Feature engineering module** (rolling stats, cluster normalization)

2. **C-MAPSS RUL Prediction Pipeline**:
   - Cluster normalization (regime-aware baselines)
   - Extended rolling windows [5, 10, 20, 30, 50]
   - Stacking ensemble (LGB + GBR + XGB + Ridge)
   - **Best results: FD001 13.10 RMSE, FD002 14.15 RMSE**

3. **Complete Documentation**:
   - API reference for all engines
   - Rolling window statistics reference
   - Cluster normalization guide
   - Ablation study results
   - Stacking ensemble analysis
   - Experiment log
   - Reproducible code scripts

## Best Configuration by Dataset

### FD001 (Single Operating Condition)
```python
# Use stacking ensemble
model = StackingRegressor(
    estimators=[
        ('lgb', LGBMRegressor(...)),
        ('gbr', GradientBoostingRegressor(...)),
        ('xgb', XGBRegressor(...)),
    ],
    final_estimator=Ridge(alpha=1.0),
)
# Result: 13.10 RMSE (gap to SOTA: +21.1%)
```

### FD002 (Six Operating Conditions)
```python
# Use XGBoost alone
model = XGBRegressor(
    n_estimators=500, max_depth=6, learning_rate=0.02,
    subsample=0.8, colsample_bytree=0.7,
)
# Result: 14.15 RMSE (gap to SOTA: +23.5%)
```

### Common Configuration
```python
WINDOWS = [5, 10, 20, 30, 50]  # Extended windows
N_CLUSTERS = {'FD001': 1, 'FD002': 6}
HEALTHY_PCT = 0.20
```

## Key Insights

1. **Regime-aware baselines are essential** for multi-condition datasets
2. **Window 50 captures long-term degradation** - most important feature
3. **Window 5 captures rapid changes** - complements long-term trends
4. **hd_slope_50 is #1 feature** - 30% of model importance
5. **Keep all features** - XGBoost handles selection internally
6. **Stacking helps single-condition datasets** - FD001 improved 1.2%
7. **XGBoost alone best for multi-condition** - FD002 doesn't benefit from stacking

## Results Summary

| Dataset | Best Model | RMSE | SOTA | Gap |
|---------|------------|------|------|-----|
| FD001 | Stacking | **13.10** | 10.82 | +21.1% |
| FD002 | XGBoost | **14.15** | 11.46 | +23.5% |

---

**Package Repository:** https://github.com/prism-engines/diagnostics  
**Version:** 0.1.0  
**License:** MIT  
**Last Updated:** January 21, 2026

---

# Summary

## What We Built

1. **orthon package** (v0.1.0): Pip-installable behavioral geometry engine with:
   - 11 vector engines (Hurst, entropy, GARCH, wavelets, etc.)
   - 9 geometry engines (PCA, clustering, MST, etc.)
   - 7 state engines (Granger, DTW, DMD, etc.)
   - 5 temporal dynamics engines
   - 3 observation-level engines
   - **Feature engineering module** (rolling stats, cluster normalization)

2. **C-MAPSS RUL Prediction Pipeline**:
   - Cluster normalization (regime-aware baselines)
   - Extended rolling windows [5, 10, 20, 30, 50]
   - **Best results: FD001 13.26 RMSE, FD002 14.15 RMSE**

3. **Complete Documentation**:
   - API reference for all engines
   - Rolling window statistics reference
   - Cluster normalization guide
   - Ablation study results
   - Experiment log
   - Reproducible code scripts

## Best Configuration

```python
WINDOWS = [5, 10, 20, 30, 50]  # Extended windows
N_CLUSTERS = {'FD001': 1, 'FD002': 6}
HEALTHY_PCT = 0.20

XGB_PARAMS = {
    'n_estimators': 500,
    'max_depth': 6,
    'learning_rate': 0.02,
    'subsample': 0.8,
    'colsample_bytree': 0.7,
}
```

## Key Insights

1. **Regime-aware baselines are essential** for multi-condition datasets
2. **Window 50 captures long-term degradation** - most important feature
3. **Window 5 captures rapid changes** - complements long-term trends
4. **hd_slope_50 is #1 feature** - 30% of model importance
5. **Keep all features** - XGBoost handles selection internally

---

**Package Repository:** https://github.com/prism-engines/diagnostics  
**Version:** 0.1.0  
**License:** MIT  
**Last Updated:** January 20, 2026